# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [20]:
import syft as sy

In [3]:
hook = sy.TorchHook(th)

In [4]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [5]:
bob = sy.VirtualWorker(hook, id="bob")

In [6]:
bob._objects

{}

In [7]:
x = th.tensor([1,2,3,4,5])

In [8]:
x = x.send(bob)

In [9]:
bob._objects

{59007095310: tensor([1, 2, 3, 4, 5])}

In [10]:
x.location

<VirtualWorker id:bob #objects:1>

In [11]:
x.id_at_location

59007095310

In [12]:
x.id

72172102243

In [13]:
x.owner

<VirtualWorker id:me #objects:0>

In [14]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [15]:
x

(Wrapper)>[PointerTensor | me:72172102243 -> bob:59007095310]

In [16]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [17]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [ ]:
# try this project here!

In [18]:
y = th.tensor([5,4,3,2,1])

In [22]:
alice = sy.VirtualWorker(hook, id="alice")
carrie = sy.VirtualWorker(hook, id="carrie")

In [23]:
y = y.send(alice, carrie)

In [33]:
y.child.child

{'alice': (Wrapper)>[PointerTensor | me:83663868544 -> alice:72878431180],
 'carrie': (Wrapper)>[PointerTensor | me:89686059890 -> carrie:72211636372]}

# Lesson: Introducing Remote Arithmetic

In [34]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [37]:
x

(Wrapper)>[PointerTensor | me:82065482827 -> bob:35754443077]

In [38]:
y

(Wrapper)>[PointerTensor | me:24500477308 -> bob:11408977311]

In [39]:
z = x + y

In [41]:
z

(Wrapper)>[PointerTensor | me:30054420835 -> bob:17351312444]

In [42]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [43]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:32123697739 -> bob:48166557036]

In [44]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [45]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [46]:
z = (x + y).sum()

In [47]:
z.backward()

(Wrapper)>[PointerTensor | me:92148025724 -> bob:75197516946]

In [48]:
x = x.get()

In [49]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [50]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [71]:
# try this project here!
input = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1.],[0.],[0.]], requires_grad=True).send(bob)

In [72]:
weights = th.tensor([[0.], [0.]], requires_grad=True).send(bob)

In [73]:
for i in range (15):
    pred = input.mm(weights)
    loss = ((pred - target)**2).sum()
    
    loss.backward()
    
    weights.data.sub_(weights.grad * 0.1)
    weights.grad += 0
    
    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.3872)
tensor(1.5484)
tensor(1.5509)
tensor(0.5217)
tensor(0.8480)
tensor(2.0888)
tensor(1.7037)
tensor(0.2423)
tensor(0.4025)
tensor(1.7419)
tensor(1.6684)
tensor(0.6213)
tensor(0.9553)


# Lesson: Garbage Collection and Common Errors


In [74]:
bob = bob.clear_objects()

In [75]:
bob._objects

{}

In [76]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [77]:
bob._objects

{6638631994: tensor([1, 2, 3, 4, 5])}

In [78]:
del x

In [79]:
bob._objects

{}

In [80]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [81]:
bob._objects

{27481429630: tensor([1, 2, 3, 4, 5])}

In [82]:
x = "asdf"

In [83]:
bob._objects

{}

In [84]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [85]:
x

(Wrapper)>[PointerTensor | me:88478293926 -> bob:73181488430]

In [86]:
bob._objects

{73181488430: tensor([1, 2, 3, 4, 5])}

In [87]:
x = "asdf"

In [88]:
bob._objects

{73181488430: tensor([1, 2, 3, 4, 5])}

In [89]:
del x

In [90]:
bob._objects

{73181488430: tensor([1, 2, 3, 4, 5])}

In [91]:
bob = bob.clear_objects()
bob._objects

{}

In [92]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [93]:
bob._objects

{83618923109: tensor([1, 2, 3, 4, 5])}

In [94]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [95]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually located on another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:35655563935 -> bob:96081685307]
Tensor B: tensor([1, 1, 1, 1, 1])

In [96]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [97]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #objects:2> while the other is on <VirtualWorker id:alice #objects:2>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [98]:
from torch import nn, optim

In [99]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [100]:
# A Toy Model
model = nn.Linear(2,1)

In [101]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [102]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(5.3090)
tensor(0.6418)
tensor(0.2110)
tensor(0.1260)
tensor(0.0845)
tensor(0.0579)
tensor(0.0400)
tensor(0.0279)
tensor(0.0196)
tensor(0.0139)
tensor(0.0099)
tensor(0.0071)
tensor(0.0051)
tensor(0.0037)
tensor(0.0027)
tensor(0.0020)
tensor(0.0015)
tensor(0.0011)
tensor(0.0008)
tensor(0.0006)


In [103]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [104]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [105]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [106]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [107]:
train()

tensor(0.7045, requires_grad=True)
tensor(0.3611, requires_grad=True)
tensor(0.1425, requires_grad=True)
tensor(0.2482, requires_grad=True)
tensor(0.0817, requires_grad=True)
tensor(0.1609, requires_grad=True)
tensor(0.0526, requires_grad=True)
tensor(0.1052, requires_grad=True)
tensor(0.0348, requires_grad=True)
tensor(0.0696, requires_grad=True)
tensor(0.0234, requires_grad=True)
tensor(0.0466, requires_grad=True)
tensor(0.0161, requires_grad=True)
tensor(0.0316, requires_grad=True)
tensor(0.0112, requires_grad=True)
tensor(0.0217, requires_grad=True)
tensor(0.0080, requires_grad=True)
tensor(0.0150, requires_grad=True)
tensor(0.0057, requires_grad=True)
tensor(0.0105, requires_grad=True)
tensor(0.0041, requires_grad=True)
tensor(0.0074, requires_grad=True)
tensor(0.0030, requires_grad=True)
tensor(0.0053, requires_grad=True)
tensor(0.0022, requires_grad=True)
tensor(0.0038, requires_grad=True)
tensor(0.0017, requires_grad=True)
tensor(0.0027, requires_grad=True)
tensor(0.0012, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [108]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [109]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [110]:
x = x.send(alice)

In [111]:
bob._objects

{3737407877: tensor([1, 2, 3, 4, 5])}

In [112]:
alice._objects

{35999865031: (Wrapper)>[PointerTensor | alice:35999865031 -> bob:3737407877]}

In [113]:
y = x + x

In [114]:
y

(Wrapper)>[PointerTensor | me:41951995667 -> alice:7778323128]

In [115]:
bob._objects

{3737407877: tensor([1, 2, 3, 4, 5]),
 21633424152: tensor([ 2,  4,  6,  8, 10])}

In [116]:
alice._objects

{35999865031: (Wrapper)>[PointerTensor | alice:35999865031 -> bob:3737407877],
 7778323128: (Wrapper)>[PointerTensor | alice:7778323128 -> bob:21633424152]}

In [117]:
jon = sy.VirtualWorker(hook, id="jon")

In [118]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [119]:
bob._objects

{22577254141: tensor([1, 2, 3, 4, 5])}

In [120]:
alice._objects

{50297282605: (Wrapper)>[PointerTensor | alice:50297282605 -> bob:22577254141]}

In [121]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:50297282605 -> bob:22577254141]

In [122]:
bob._objects

{22577254141: tensor([1, 2, 3, 4, 5])}

In [123]:
alice._objects

{}

In [124]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [125]:
bob._objects

{}

In [126]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [127]:
bob._objects

{52083401684: tensor([1, 2, 3, 4, 5])}

In [128]:
alice._objects

{30409645431: (Wrapper)>[PointerTensor | alice:30409645431 -> bob:52083401684]}

In [129]:
del x

In [130]:
bob._objects

{}

In [131]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [132]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [133]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [134]:
bob._objects

{44947911398: tensor([1, 2, 3, 4, 5])}

In [135]:
alice._objects

{}

In [136]:
x.move(alice)

(Wrapper)>[PointerTensor | me:14878287770 -> alice:14878287770]

In [137]:
bob._objects

{}

In [138]:
alice._objects

{14878287770: tensor([1, 2, 3, 4, 5])}

In [139]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [140]:
bob._objects

{6206100818: tensor([1, 2, 3, 4, 5])}

In [141]:
alice._objects

{14878287770: tensor([1, 2, 3, 4, 5]),
 331100355: (Wrapper)>[PointerTensor | alice:331100355 -> bob:6206100818]}

In [142]:
x.remote_get()

(Wrapper)>[PointerTensor | me:84576921925 -> alice:331100355]

In [143]:
bob._objects

{}

In [144]:
alice._objects

{14878287770: tensor([1, 2, 3, 4, 5]), 331100355: tensor([1, 2, 3, 4, 5])}

In [145]:
x.move(bob)

(Wrapper)>[PointerTensor | me:84576921925 -> bob:84576921925]

In [146]:
x

(Wrapper)>[PointerTensor | me:84576921925 -> bob:84576921925]

In [147]:
bob._objects

{84576921925: tensor([1, 2, 3, 4, 5])}

In [148]:
alice._objects

{14878287770: tensor([1, 2, 3, 4, 5])}